<a href="https://colab.research.google.com/github/Texsic/Machine-Learning/blob/main/shotfinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OPTIONAL: To test your solution against a subset of our test cases, you can [install the `otter` library](https://otter-grader.readthedocs.io/en/latest/index.html#installation) in your local python environment with your choice of the commands below and run the next cell:
```
pipx install otter-grader
# or
pip install otter-grader
```

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("shot_finder.ipynb")
# NOTE: If you change the name of this file, you will also need to update the line above to use Otter to test your solution.

**Question:** Given a list of shot dictionaries, implement `find_faulty_shots` to return a list of unique shotIDs for shots labeled incorrectly, in order of most recent to oldest.

Each shot dictionary contains the following fields:
- shotID: str
- period: int
- shotClockSeconds: float
- gameClockSeconds: float
- fg2Attempted: int
- fg2Made: int
- fg3Attempted: int
- fg3Made: int
- ftAttempted: int
- ftMade: int
- points: int

All field values are guaranteed to be correct except for shotClockSeconds, gameClockSeconds, and points.

A shot is labeled incorrectly if any of the following are true:
- It has an invalid shot clock value.
- It has an invalid game clock value. Note that gameClockSeconds represents the number of seconds remaining in the current period. A period value of 1-4 represents regulation and a period value of 5 or more represents overtime.
- It has an invalid points value, relative to the points expected based on shot type/zone, whether the shot was made, and the inclusion of free throws.

Each shotID is structured as MMDDYYYY_GG_FFFFFFFF, where:
- MMDDYYYY = date of the game
- GG = order in which the game was played relative to other games that day (e.g. 01, 02). You can assume that 02 starts after 01 finishes.
- FFFFFFFF = frame number of the shot within that game, left-padded to 8 digits (e.g. 00000001 for the first shot).

You may assume standard NBA rules for the shot clock, game clock, and point values.

In [ ]:
from datetime import datetime
# DO NOT IMPORT ANY ADDITIONAL LIBRARIES

def find_faulty_shots(shots_array: list[dict]) -> list[str]:

    # Helper function defined inside for safety and encapsulation
    def parse_shot_id(shot_id):
        parts = shot_id.split('_')
        date_str = parts[0]
        # Return tuple (Year, Month, Day, Game, Frame)
        return (date_str[4:8], date_str[0:2], date_str[2:4], int(parts[1]), int(parts[2]))

    faulty_shot_ids = set()

    for shot in shots_array:
        is_faulty = False

        # 1. Validate Shot Clock
        if not (0 <= shot['shotClockSeconds'] <= 24):
            is_faulty = True

        # 2. Validate Game Clock
        # Regulation (Periods 1-4) = 720s, Overtime (5+) = 300s
        max_seconds = 720 if shot['period'] <= 4 else 300
        if not (0 <= shot['gameClockSeconds'] <= max_seconds):
            is_faulty = True

        # 3. Validate Points
        expected_points = (shot['fg2Made'] * 2 + shot['fg3Made'] * 3 + shot['ftMade'])
        if shot['points'] != expected_points:
            is_faulty = True

        if is_faulty:
            faulty_shot_ids.add(shot['shotID'])

    # Sort and return
    return sorted(list(faulty_shot_ids), key=parse_shot_id, reverse=True)

The cell below will run a subset of our tests using the `grader` generated in the cells above your implementation, but it will only work if you've installed the `otter` library.

In [ ]:
grader.check("q1")